In [2]:
import numpy as np
import pandas as pd

In [3]:
fourrage = pd.read_excel("../INRA2018_TablesFourrages_etude_prediction_20241121.xlsx", 
                        engine="openpyxl", header = 1) #, dtype = {'Libellé 4' : 'string'})

In [3]:
fourrage.head(2)

,No,Etat,Code INRA,Libellé 0,Libellé 1,Libellé 2,Libellé 3,Libellé 4,MS,UFL,...,GlyDI,AlaDI,TyrDI,C14:0,C16:0,C16:1,C18:0,C18:1,C18:2,C18:3
0,1,1,FV0010,FOURRAGES VERTS,"PRAIRIES PERMANENTES, PLAINE (NORMANDIE)",1er cycle,"15-25 avril, déprimage, ST = 172°C",<NA>,15.5,1.07,...,6.94,6.87,4.13,1.0,15.68,1.4,2.30,4.04,16.34,54.50
1,2,1,FV0020,FOURRAGES VERTS,"PRAIRIES PERMANENTES, PLAINE (NORMANDIE)",1er cycle,"1-10 mai, pâturage, ST = 298°C",<NA>,16.6,1.02,...,6.93,6.88,4.14,1.0,16.36,1.4,2.38,4.19,16.89,52.75


In [10]:
type(fourrage.loc[1,'Libellé 4'])

float

In [4]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.multioutput import MultiOutputRegressor
import prince 

%load_ext autoreload
%autoreload 1
%aimport utils

text_cols = ['Libellé 0', 'Libellé 1', 'Libellé 2' ,'Libellé 3', 'Libellé 4']
num_cols = ['MS', 'MM', 'MAT', 'CB', 'NDF', 'ADF', 'EE']
targets = ['UFL', 'UFV', 'PDIA','PDI', 'BPR']

#X = fourrage[cat_cols]
#print(utils.SplitTransformer().fit(X).transform(X)['Libellé 0-1'].unique())
#bug identified, only 1 comma in first columns...


text_pipeline = Pipeline([
    ('spliting libelle', utils.SplitTransformer()),
    ("one hot encoding", OneHotEncoder(sparse_output=False, handle_unknown='ignore')),
    #("MCA",prince.MCA())
])

num_pipeline = Pipeline([
    ('num_standardization', StandardScaler()),
    ('num_imputation', KNNImputer())    
]) 

preprocessing = ColumnTransformer([
    ("categorical_preproc", text_pipeline, text_cols),
    ("num_preproc", num_pipeline, num_cols)
])

model = Pipeline([
    ('preprocessing', preprocessing),
    ("regressor", MultiOutputRegressor(GradientBoostingRegressor(random_state=42), n_jobs=2) )    
])


text_pipeline.set_output(transform='pandas')


Pipeline(steps=[('spliting libelle', SplitTransformer()),
                ('one hot encoding',
                 OneHotEncoder(handle_unknown='ignore', sparse_output=False))])

In [5]:
model

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('categorical_preproc',
                                                  Pipeline(steps=[('spliting '
                                                                   'libelle',
                                                                   SplitTransformer()),
                                                                  ('one hot '
                                                                   'encoding',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['Libellé 0', 'Libellé 1',
                                                   'Libellé 2', 'Libellé 3',
                                                   'Libellé 4']),
                                                 ('num_preproc',
                                                  Pipeline(steps=[('num_standardization',
                                                                   StandardScaler()),
                                                                  ('num_imputation',
                                                                   KNNImputer())]),
                                                  ['MS', 'MM', 'MAT', 'CB',
                                                   'NDF', 'ADF', 'EE'])])),
                ('regressor',
                 MultiOutputRegressor(estimator=GradientBoostingRegressor(random_state=42),
                                      n_jobs=2))])

# Train-test split

In [6]:
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(fourrage[text_cols+num_cols], fourrage[targets])

In [11]:
X_train['Libellé 4'].unique()[3]


'Début floraison'

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('categorical_preproc',
                                                  Pipeline(steps=[('spliting '
                                                                   'libelle',
                                                                   SplitTransformer()),
                                                                  ('one hot '
                                                                   'encoding',
                                                                   OneHotEncoder(sparse_output=False))]),
                                                  ['Libellé 0', 'Libellé 1',
                                                   'Libellé 2', 'Libellé 3',
                                                   'Libellé 4']),
                                                 ('num_preproc',
                                                  Pipeline(steps=[('num_standardization',
                                                                   StandardScaler()),
                                                                  ('num_imputation',
                                                                   KNNImputer())]),
                                                  ['MS', 'MM', 'MAT', 'CB',
                                                   'NDF', 'ADF', 'EE'])])),
                ('regressor', GradientBoostingRegressor(random_state=42))])

# Model saving

In [7]:
from pickle import dump

model.fit(X_train, y_train)
with open('baseline.pkl', 'wb') as f:
    dump(model, f, protocol=5)